ETL Pipeline for Two Datasets

Here’s a medium difficulty version:

Extract both CSVs.

Transform:

Sales → clean + add total_amount.

Customers → clean + normalize location names.

Load into the same SQLite DB but different tables (sales, customers).

Logging + error handling throughout.

In [13]:
import os
import pandas as pd
import sqlite3
import logging


# define the file paths for the sales and customers data directory.
base_dir = r"D:\DE\ETL_Practices\Practice_Coding\8 Weeks Pyhton Practice for DE\Week4_Data Pipelines in Pure Python\Sample Dataset"

sales_path = os.path.join(base_dir, "sales_data.csv")
customers_path = os.path.join(base_dir, "customers_data.csv")


In [ ]:
# Configure Logging

logging.basicConfig(
    filename="etl_pipeline_day6_extended.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)


# Extract Function

def extract(file_path: str) -> pd.DataFrame:
    """Extract data from CSV into a DataFrame"""
    try:
        df = pd.read_csv(file_path)
        logging.info(f" Extracted {file_path} successfully")
        return df
    except Exception as e:
        logging.error(f" Extraction failed for {file_path}: {e}")
        return pd.DataFrame()


# Transform Functions

def transform_sales(df: pd.DataFrame) -> pd.DataFrame:
    """Transform sales dataset"""
    try:
        if df.empty:
            logging.warning(" Sales data empty")
            return df

        df = df.drop_duplicates()
        df = df.fillna({"customer_name": "Unknown"})
        df["total_amount"] = df["quantity"] * df["price"]

        logging.info(" Sales transformation successful")
        return df
    except Exception as e:
        logging.error(f" Sales transformation failed: {e}")
        return pd.DataFrame()

def transform_customers(df: pd.DataFrame) -> pd.DataFrame:
    """Transform customers dataset"""
    try:
        if df.empty:
            logging.warning(" Customers data empty")
            return df

        df = df.drop_duplicates()
        df = df.fillna({"location": "Unknown"})
        df["location"] = df["location"].str.title()

        logging.info(" Customers transformation successful")
        return df
    except Exception as e:
        logging.error(f" Customers transformation failed: {e}")
        return pd.DataFrame()

def join_sales_customers(sales_df: pd.DataFrame, customers_df: pd.DataFrame) -> pd.DataFrame:
    """Join sales and customers to build fact table"""
    try:
        if sales_df.empty or customers_df.empty:
            logging.warning(" One or both datasets are empty, cannot join")
            return pd.DataFrame()

        fact_df = sales_df.merge(
            customers_df,
            on="customer_name",
            how="left"  # keep all sales even if customer missing
        )

        logging.info(" Joined sales and customers successfully")
        return fact_df
    except Exception as e:
        logging.error(f" Join failed: {e}")
        return pd.DataFrame()


# Load Function

def load(df: pd.DataFrame, db_name: str, table_name: str):
    """Load DataFrame into SQLite DB"""
    try:
        if df.empty:
            logging.warning(f" No data to load into {table_name}")
            return

        conn = sqlite3.connect(db_name)
        df.to_sql(table_name, conn, if_exists="replace", index=False)
        conn.close()

        logging.info(f" Loaded data into {table_name} table successfully")
    except Exception as e:
        logging.error(f"Load failed for {table_name}: {e}")


# Main ETL Pipeline

if __name__ == "__main__":
    logging.info(" Day 6 Extended ETL pipeline started")

    # Extract
    sales_df = extract(sales_path)
    customers_df = extract(customers_path)
    print(f"sales_df is \n {sales_df.head()}")
    print(f"customers_df is \n {customers_df.head()}")

    # Transform
    clean_sales = transform_sales(sales_df)
    clean_customers = transform_customers(customers_df)
    print(f"clean_sales is \n {clean_sales.head()}")
    print(f"clean_customers is \n {clean_customers.head()}")

    # Join
    fact_sales_table = join_sales_customers(clean_sales, clean_customers)
    print(f"fact_sales_table is \n {fact_sales_table.head()}")

    # Load into DB (3 tables)
    load(clean_sales, "day6_extended_etl.db", "sales")
    load(clean_customers, "day6_extended_etl.db", "customers")
    load(fact_sales_table, "day6_extended_etl.db", "fact_sales")
    

    logging.info("🏁 Day 6 Extended ETL pipeline finished")
    



sales_df is 
    order_id  order_date customer_name     product  quantity  price
0      1001  2025-01-01      John Doe      Laptop         1   1200
1      1002  2025-01-02    Jane Smith       Phone         2    600
2      1003  2025-01-03       Bob Lee  Headphones         3    100
3      1004  2025-01-03      Anna Kim      Laptop         1   1200
4      1005  2025-01-04      John Doe       Phone         1    600
customers_df is 
   customer_id customer_name             email   location
0        C001      John Doe  john@example.com        USA
1        C002    Jane Smith  jane@example.com     Canada
2        C003       Bob Lee   bob@example.com         UK
3        C004      Anna Kim  anna@example.com      Korea
4        C005     Mike Chan  mike@example.com  Singapore
cleasn_sales is 
    order_id  order_date customer_name     product  quantity  price  \
0      1001  2025-01-01      John Doe      Laptop         1   1200   
1      1002  2025-01-02    Jane Smith       Phone         2    600